In [1]:
import pandas as pd

import os

import numpy as np

import pickle

import ast

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
nlp = spacy.load("en_core_web_md")

from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.display import clear_output

In [2]:
df_videos_details = pd.read_csv('data/get_videos_details/videos_details.csv')

In [3]:
df_videos_details

,video_id,title,description,category_id,relevant_topic_ids,relevant_topic_categories
0,WyUyJKm-MiY,Reacting to MEAN Tik Toks About Me (This Cross...,Today we are diving into the OTHER side of Tik...,24,NaN,NaN
1,5mUQo6lQG-U,Why My New Palette is CANCELLED... and Reveali...,"Hey everyone, welcome BACK to my channel! Toda...",26,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
2,QhbR5TEAMco,Meet My 2 NEW Pomeranian Puppies! 🐶,Hey everyone!!! Today I'm so excited to introd...,26,"['/m/019_rr', '/m/068hy', '/m/019_rr', '/m/068...",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
3,3qYykmHx5gc,Reacting To Tik Toks About Me (They Broke In!),Follow my official Tik Tok account: https://ww...,24,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
4,SxixuClI2vA,Sailor Moon x ColourPop Makeup… Is It Jeffree ...,Hey everyone... Welcome BACK to my channel! To...,26,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
...,...,...,...,...,...,...
1595,A-ZXP9vN7hg,"Living In Norway, Underwater Restaurant, New R...",Hi Friends!!\nHere is a lil blog of what we’ve...,26,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
1596,LYHs9j3-bG4,I Tried Dark Contact Lenses For the First Time...,Hi Friends!!\nI got a bunch of requests to fil...,26,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
1597,qrWkDwpqAEI,NEW Pat McGrath Full Foundation Review + Naomi...,Hey baes!!!\nFor todays video I wanted to recr...,26,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...
1598,6OUQi5ymq7o,Learning To Love Myself | GRWM,Hey babes!!\nFor todays video I’m sharing with...,26,"['/m/019_rr', '/m/019_rr']",['https://en.wikipedia.org/wiki/Lifestyle_(soc...


In [4]:
topics = dict()
with open('data/youtube_topicIDs.txt', 'r') as f:
    for line in f.readlines():
        key, value = line.split(',')[0], line.split(',')[1][:-1]
        topics.update({key:value})

np.save('data/topic_ids.npy', topics, allow_pickle=True)
#t = np.load('data/topic_ids.npy',allow_pickle=True).tolist()

In [10]:
keywords = []
with open('data/key_words.txt', 'r') as f:
    for line in f.readlines():
        keywords.append(line.strip().lower().split(','))

np.save('data/keywords.npy', keywords, allow_pickle=True)

In [12]:
makeup_brands = []
with open('data/makeup_brands.txt', 'r') as f:
    for line in f.readlines():
        makeup_brands.append(line.strip().lower().split(','))

np.save('data/makeup_brands.npy', makeup_brands, allow_pickle=True)

In [7]:
categories = dict()
with open('data/youtube_categoryIDs.txt', 'r') as f:
    for line in f.readlines():
        key, value = line.split('-')[0].strip(), line.split('-')[1][:-1].strip()
        categories.update({key:value})


np.save('data/categories_ids.npy', categories, allow_pickle=True)
#t = np.load('data/topic_ids.npy',allow_pickle=True).tolist()

In [8]:
def tokenizer(sentence):
    arr = []
    for w in sentence.split(' '):
        arr.append(w.translate(str.maketrans('', '', string.punctuation)).strip().lower())
    return arr

In [13]:
def find_topics(title, relevant_topics):
    title_topics = set()
    other_topics = set()
    for keyword, generic_keyword in keywords:
        if len(keyword.split(' '))>1:
            matches = keyword.strip().split(' ')
            if all(x in title.lower().split(' ') for x in matches):
                title_topics.add(generic_keyword)
        else:
            if keyword in tokenizer(title):
                title_topics.add(generic_keyword)
    if relevant_topics != None:
        for t in relevant_topics:
            if t == 'Lifestyle_(sociology)':
                other_topics.add('lifestyle')
            else:
                other_topics.add(t.lower())
        total_topics = title_topics.union(other_topics)
        return total_topics
    else:
        return title_topics

In [18]:
def find_brands(description):
    cited_brand = set()
    for brand in makeup_brands:
        original = brand[-1]
        matches = brand[0].split(' ')
        if all(x in description.lower().split(' ') for x in matches):
            cited_brand.add(original)
    return cited_brand

In [30]:
def compute_details(df):
    df_arr_computed = []
    for idx, row in enumerate(df.values):
        print('Video: {}/{}'.format(idx,len(df.values)))
        clear_output(wait=True)
        relevant_topics_cat=None
        line=[row[0],row[1],row[2]]
        try:
            line.append(categories[str(row[3])])
        except:
            line.append(None)
        try:
            l = row[4]
            l = ast.literal_eval(l)
            l = [n.strip() for n in l]
            tps = [topics[item] for item in l]
            line.append(tps)
        except:
            line.append(None)
        try:
            v = row[5]
            v = ast.literal_eval(v)
            v = [n.split('/')[-1] for n in v]
            relevant_topics_cat = v
            line.append(v)
        except:
            line.append(None)
        #vedo se il titolo contiene keywords
        if type(row[1]) is str:
            computed_topics = find_topics(row[1], relevant_topics_cat if relevant_topics_cat!=None else [])
            line.append(list(computed_topics))
        else:
            line.append([])
        #vedo se la descrizione contiene brand
        if type(row[2]) is str:
            cited_brand = find_brands(row[2])
            line.append(list(cited_brand))
        else:
            line.append([])
        df_arr_computed.append(line)
    c =[item for item in df.columns]
    c.append('topics')
    c.append('brands')
    df_computed = pd.DataFrame(df_arr_computed, columns = c)
    return df_computed

In [31]:
df_computed_all = compute_details(df_videos_details)

Video: 1599/1600


In [32]:
df_computed_all

,video_id,title,description,category_id,relevant_topic_ids,relevant_topic_categories,topics,brands
0,WyUyJKm-MiY,Reacting to MEAN Tik Toks About Me (This Cross...,Today we are diving into the OTHER side of Tik...,Entertainment,None,None,"[social media, review]",[jeffree star cosmetics]
1,5mUQo6lQG-U,Why My New Palette is CANCELLED... and Reveali...,"Hey everyone, welcome BACK to my channel! Toda...",Howto & Style,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],"[new product launch, lifestyle, review]",[jeffree star cosmetics]
2,QhbR5TEAMco,Meet My 2 NEW Pomeranian Puppies! 🐶,Hey everyone!!! Today I'm so excited to introd...,Howto & Style,"[Lifestyle, Pets, Lifestyle, Pets]","[Lifestyle_(sociology), Pet]","[lifestyle, pet]",[]
3,3qYykmHx5gc,Reacting To Tik Toks About Me (They Broke In!),Follow my official Tik Tok account: https://ww...,Entertainment,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],"[lifestyle, social media, review]",[]
4,SxixuClI2vA,Sailor Moon x ColourPop Makeup… Is It Jeffree ...,Hey everyone... Welcome BACK to my channel! To...,Howto & Style,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],"[lifestyle, review]","[colourpop cosmetics, jeffree star cosmetics]"
...,...,...,...,...,...,...,...,...
1595,A-ZXP9vN7hg,"Living In Norway, Underwater Restaurant, New R...",Hi Friends!!\nHere is a lil blog of what we’ve...,Howto & Style,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],"[lifestyle, private life/information]",[mac cosmetics]
1596,LYHs9j3-bG4,I Tried Dark Contact Lenses For the First Time...,Hi Friends!!\nI got a bunch of requests to fil...,Howto & Style,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],[lifestyle],[mac cosmetics]
1597,qrWkDwpqAEI,NEW Pat McGrath Full Foundation Review + Naomi...,Hey baes!!!\nFor todays video I wanted to recr...,Howto & Style,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],"[tutorial, lifestyle, review]","[mac cosmetics, pat mcgrath]"
1598,6OUQi5ymq7o,Learning To Love Myself | GRWM,Hey babes!!\nFor todays video I’m sharing with...,Howto & Style,"[Lifestyle, Lifestyle]",[Lifestyle_(sociology)],"[lifestyle, get ready with me]","[mac cosmetics, urban decay]"


In [35]:
df_computed_reduced = df_computed_all[df_computed_all.columns[[0,3,6,7]]]
df_computed_reduced

,video_id,category_id,topics,brands
0,WyUyJKm-MiY,Entertainment,"[social media, review]",[jeffree star cosmetics]
1,5mUQo6lQG-U,Howto & Style,"[new product launch, lifestyle, review]",[jeffree star cosmetics]
2,QhbR5TEAMco,Howto & Style,"[lifestyle, pet]",[]
3,3qYykmHx5gc,Entertainment,"[lifestyle, social media, review]",[]
4,SxixuClI2vA,Howto & Style,"[lifestyle, review]","[colourpop cosmetics, jeffree star cosmetics]"
...,...,...,...,...
1595,A-ZXP9vN7hg,Howto & Style,"[lifestyle, private life/information]",[mac cosmetics]
1596,LYHs9j3-bG4,Howto & Style,[lifestyle],[mac cosmetics]
1597,qrWkDwpqAEI,Howto & Style,"[tutorial, lifestyle, review]","[mac cosmetics, pat mcgrath]"
1598,6OUQi5ymq7o,Howto & Style,"[lifestyle, get ready with me]","[mac cosmetics, urban decay]"


In [36]:
df_computed_all.to_csv('data/get_videos_details/videos_details_computed_complete.csv', index = False)
df_computed_reduced.to_csv('data/get_videos_details/videos_details_computed.csv', index = False)